# Курсовой проект

По дисциплине "Интеллектуальный анализ и обработка данных".

Выполнили: ст. гр. 4230М А.Н. Секунов, Д.А. Фомин.


## Задание

1.
    1. Для вариантов 1-20 дан многомерный набор данных телеметрической информации (ТМИ) малого космического аппарата, полученный в результате  предобработки и разметки в ЛР3. Разработать приложение на языке Python и  необходимых библиотек машинного и глубокого обучения на основе  классификационной модели, заданной в варианте, которое определяет техническое  состояние малого космического аппарата: штатное ф-е, отказ, сбой, на основе  данных его ТМИ.
    2. Для вариантов 21-40 дан многомерный набор данных сетевого трафика. Разработать приложение на языке Python и необходимых библиотек машинного и глубокого обучения на основе классификационной модели, заданной в варианте, х, которое определяет попытки сетевых вторжений на основе данных сетевого трафика.
2. Выполнить разведочный анализ данных как в ЛР.
3. Анализ выполняется для исходного набора данных и преобразованного исходного набора данных, который Вы подбираете самостоятельно с целью получения лучшей точности классификации.
4. Проверить набор данных на сбалансированность классов. В случае несбалансированного набора данных помимо “Classification report” с основными метриками: Accuracy, Precision, Recall, F1-score, также оценивать balanced_accuracy_score. Также оценивать и анализировать ROC_AUC_score.
5. Для выбранной комбинации набора данных и предиктивной модели выполняется поиск гиперпараметров с помощью генетического алгоритма. Реализовывать ГА можно самостоятельно или воспользоваться библиотекой DEAP (https://deap.readthedocs.io/en/master/index.html). 
  Для выбранной комбинации набора данных и предиктивной модели выполняется поиск гиперпараметров методом greedsearch. Сравнить полученные результаты.
  Во все НС классификационные модели добавить слой регуляризации Dropuot().
  Сохранять лог всех генерируемых НС моделей в популяции.
  Гиперпараметры: число сверточных фильтров в сверточных слоях, размер свертки в сверточных слоях, число нейронов в рекуррентных слоях,  число нейронов в полносвязных слоях, параметр в слое Dropuot(), алгоритм обучения.
6. На неразмеченной части набора данных выполнить предобучение с помощью автокодировщика, построенного на основании базового нейросетевого классификатора, заданного в варинте. Выполнить точную настройку (дообучение) базовой нейронной сети на размеченном наборе данных.
7. Сделать выводы по проведенному анализу.

**Вариант 28**

Гибридная нейросетевая модель: последовательностное соединение  одномерных сверточных и рекуррентных двунаправленных (Bidirectional layer)  нейросетевых слоев с полносвязным классификатором: самостоятельная реализация.

## Исходные данные

В качестве исходных данных испольщуется датасет NSL-KDD [1]. 

Используемые файлы:

 - KDDTrain+.csv - обучающая выборка с метками атаки и уровнем сложности (https://github.com/defcom17/NSL_KDD/blob/master/KDDTrain%2B.csv);
 - KDDTest+.csv - проверочное множество с метками атаки и уровнем сложностиа (https://github.com/defcom17/NSL_KDD/blob/master/KDDTest%2B.csv);
 - Field Names.csv - наименования полей (https://github.com/defcom17/NSL_KDD/blob/master/Field%20Names.csv);
 - Attack Types.csv - типы атак (https://github.com/defcom17/NSL_KDD/blob/master/Attack%20Types.csv).

Наюор данных содержит 4 категории угроз [2]:

 - Denial of Service (dos). Набор атак, в которых злоумышленник ограничивает доступ верифицированным пользователям к конкретному сервису через определенный протокол (Back, Land, Neptune, Pod, Smurf,Teardrop,Apache2, Udpstorm, Processtable, Worm);
 - Remote to Local (r2l). Набор атак, в которых злоумышленник пытается получить доступ извне к локальной машине пользователя (Guess_Password, Ftp_write, Imap, Phf, Multihop, Warezmaster, Warezclient, Spy, Xlock, Xsnoop, Snmpguess, Snmpgetattack, Httptunnel, Sendmail, Named);
 - User to Root (u2r). Набор атак, в которых злоумышленник, имея доступ к машине жертвы, пытается получить права более привилегированного пользователя (Buffer_overflow, Loadmodule, Rootkit, Perl, Sqlattack, Xterm, Ps);
 - Probe. Набор атак, в которых злоумышленник пытается получить сведения об инфраструктуре пользователя (Satan, Ipsweep, Nmap, Portsweep, Mscan, Saint).

Размерность вектра составляет 43 (41 атрибут, нименования которых хранится в файле Field Names.csv, 42-ой атрибут представляет собой тип атаки, 43-й - уровень угрозы) [2].

### Загрузка исходных данных

In [4]:
import urllib.request
import os

save_path = './datasets/download'
urls = {'train': 'https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain%2B.csv',
        'test': 'https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest%2B.csv',
        'fields': 'https://raw.githubusercontent.com/defcom17/NSL_KDD/master/Field%20Names.csv',
        'types': 'https://raw.githubusercontent.com/defcom17/NSL_KDD/master/Attack%20Types.csv'}

if not os.path.exists(save_path):
    os.mkdir(save_path)

for key in urls:
    file_name = os.path.join(save_path, key + '.csv')
    urllib.request.urlretrieve(urls[key], file_name)


### Просмотр исходных данных

#### Формирование заголовка таблицы

In [67]:
import pandas as pd

df = pd.read_csv('./datasets/download/fields.csv', header=None)
# Добавление наименований столбцов "тип атаки" и "сложность атаки"
mdf = pd.DataFrame({0: ['Attack type', 'Attack Difficulty'], 1: ['symbolic', 'continuous']})
df = df.append(mdf, ignore_index = True)
df = df.T
df = df.reindex(df.index.drop(1))

C:\Users\JustT\AppData\Local\Temp\ipykernel_19192\1459313664.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(mdf, ignore_index = True)


In [68]:
df

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack type,Attack Difficulty


#### Объединение заголовков и данных

Просмотр тестовых данных совместно со сформированными заголовками таблицы

In [69]:
data = pd.read_csv('./datasets/download/test.csv', header=None)
df = df.append(data, ignore_index = True)
data = None # Типа очистка памяти

# В этом моменте 0 строка устанавливается как заголовок столбцов,
# затем она удаляется из основной таблицы
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0))

C:\Users\JustT\AppData\Local\Temp\ipykernel_19192\3568486083.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index = True)


In [70]:
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack type,Attack Difficulty
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.0,0.0,0.0,0.0,1.0,1.0,neptune,21
2,0,tcp,private,REJ,0,0,0,0,0,0,...,0.0,0.06,0.0,0.0,0.0,0.0,1.0,1.0,neptune,21
3,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.0,0.0,0.0,0.0,normal,21
4,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.0,0.0,1.0,0.28,0.0,0.0,0.0,0.0,saint,15
5,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,smtp,SF,794,333,0,0,0,0,...,0.72,0.06,0.01,0.01,0.01,0.0,0.0,0.0,normal,21
22540,0,tcp,http,SF,317,938,0,0,0,0,...,1.0,0.0,0.01,0.01,0.01,0.0,0.0,0.0,normal,21
22541,0,tcp,http,SF,54540,8314,0,0,0,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.07,0.07,back,15
22542,0,udp,domain_u,SF,42,42,0,0,0,0,...,0.99,0.01,0.0,0.0,0.0,0.0,0.0,0.0,normal,21


### Обработка исходных данных

Требуется выполнить раделение ИД на две выборки: 

## Список использованных источников

1. Датасет NSL-KDD (https://github.com/defcom17/NSL_KDD).
2. Е.М. Бурлаков Применение метода анализа соответствий для оптимизации комбинаций атрибутов у наборов данных. Вестник ПНИПУ. Электротехника, информационные технологии, системы управления №26, 2018.